# Install package

In [2]:
# ! pip install transformers
# ! pip install simpletransformers
# ! pip install fugashi 
# ! pip install unidic_lite

# ! pip install vaderSentiment
# ! pip install emoji
# ! pip install pandas
# ! pip install smart_open
# ! pip install scikit-learn

  Using cached smart_open-6.3.0-py3-none-any.whl (56 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
# ! pip install simpletransformers

# !mamba install pytorch>=1.6 cudatoolkit=11.0 -c pytorch

In [12]:
import re
import os
import sys
import pandas as pd
from IPython.display import HTML, display

import pandas as pd
import smart_open
import numpy as np 
from sklearn import preprocessing
import emoji

from simpletransformers.classification import ClassificationModel, ClassificationArgs
from simpletransformers.classification import ClassificationModel
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()


pd.set_option('display.max_colwidth', 0)



from sklearn.model_selection import train_test_split



pd.set_option('display.max_colwidth', 0)


In [82]:
def clean_twts(tw):

  # remove urls
  tw = str(tw)
  pattern = 'https{0,1}:\/\/t.co\/[a-zA-Z0-9]+'
  tw = re.sub(pattern, "", tw)

  # remove @
  pattern = '@[a-zA-Z0-9_]+ '
  tw = re.sub(pattern, "", tw)

  # remove emojis
  tw = emoji.demojize(tw)

  return tw

def clean_twt_row(row):
  return clean_twts(row['message'])


def _correct_label(df, correct_df, text_col="message", label_col="Label"):    
    merged_df = df.merge(correct_df, left_on='message', right_on='message', how='left')
    mask_idx = merged_df['Correct_label'].isna()
    # print("Label not nan count:", (~mask_idx).sum())
    merged_df.loc[~mask_idx, 'Label'] = merged_df[~mask_idx]['Correct_label'].to_list()

    return merged_df

def correct_label(df):
    df1 = pd.read_csv(r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\bi_label_in_test_correct_labels.csv')
    df2 = pd.read_csv(r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\bi_label_in_train_correct_labels.csv')
    correct_df = pd.concat([df1, df2])
    correct_df['message'] = correct_df.apply(clean_twt_row, axis=1)
    correct_df = correct_df.rename(columns={'Label': "Correct_label"})
    
    df = _correct_label(df, correct_df=correct_df, text_col="message", label_col="Label")
    df = df.drop(columns=['Correct_label'])
    
    return df

df = pd.read_csv(r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\LIZ_negative_2021-12-20T11_42_05.000Z_2021-12-21T02_34_19.000Z_9385.csv')
df['message'] = df.apply(clean_twt_row, axis=1)
df['Label'] = 0
df = correct_label(df=df)
df
# df['Label'].sum()

,tweetid,userid,username,postdate,message,geoType,longitude,latitude,place,placeBboxwest,...,country,tweet_lang,message_en,message_cn,sentiment,topic,username_account,file,tweetid_short,Label
0,1.473120e+18,1.596093e+09,くまがいbot,2021-12-21T02:34:19.000Z,楽に死ねるあの首吊りの方法実践してくる,NaN,NaN,NaN,NaN,NaN,...,NaN,ja,NaN,NaN,0.0,NaN,kuma__bot,2021-12-20T11_42_05.000Z_2021-12-21T02_34_19.000Z_9385.csv,1473119611503900000,0.0
1,1.473120e+18,2.715222e+09,みーちゃん,2021-12-21T02:34:07.000Z,RT @badassceo: 死ぬ気でやるなよ。死んじゃうから。人の心と体は弱い。無理が続くとある日突然ポキっと折れてしまう。で、一度折れるとなかなか元に戻らないし耐久性も落ちる。人生で一番大切なのはあなたの健康だ。命だ。健康を犠牲にしてまでやる価値のある事はない。無理すんな。…,NaN,NaN,NaN,NaN,NaN,...,NaN,ja,NaN,NaN,0.0,NaN,mikibu0107,2021-12-20T11_42_05.000Z_2021-12-21T02_34_19.000Z_9385.csv,1473119560375300000,0.0
2,1.473119e+18,1.545049e+09,歌詞bot,2021-12-21T02:33:45.000Z,君のようなひとになりたいな 「僕らしいひと」になりたいな 望むならそうすりゃいいけどさ でもそれってほんとにぼくなのかい 子供騙しな夢ひとつ こんな僕なら死ねばいいのに/自傷無色,NaN,NaN,NaN,NaN,NaN,...,NaN,ja,NaN,NaN,0.0,NaN,kasi_bot_,2021-12-20T11_42_05.000Z_2021-12-21T02_34_19.000Z_9385.csv,1473119471649000000,0.0
3,1.473119e+18,1.566032e+09,ニートキラbot,2021-12-21T02:33:38.000Z,プリクラとプリキュア間違えた死にたい,NaN,NaN,NaN,NaN,NaN,...,NaN,ja,NaN,NaN,0.0,NaN,kilaneet,2021-12-20T11_42_05.000Z_2021-12-21T02_34_19.000Z_9385.csv,1473119441055800000,0.0
4,1.473119e+18,3.397248e+09,@kky10091009 詐欺師 乞食,2021-12-21T02:33:32.000Z,おはよう、ただいま、悲しい、さようなら、疲れた、おやすみなさい、こんにちは、帰宅、こんばんは、ほかいま、こんばんは、おやすみ、寝る、ほかる、死にたい、怖い、おはようございます、占って！ いってきます、ただいま、お帰り！離脱、お風呂70,NaN,NaN,NaN,NaN,NaN,...,NaN,ja,NaN,NaN,0.0,NaN,tsnjieb,2021-12-20T11_42_05.000Z_2021-12-21T02_34_19.000Z_9385.csv,1473119415235700000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8542,1.472895e+18,1.102373e+18,微笑み‼️,2021-12-20T11:42:11.000Z,死にてえ死にてえ死にてえ 漠然と死にたい,NaN,NaN,NaN,NaN,NaN,...,NaN,ja,NaN,NaN,0.0,NaN,SxFnQYMdFmUxOcT,2021-12-20T11_42_05.000Z_2021-12-21T02_34_19.000Z_9385.csv,1472895098229000000,0.0
8543,1.472895e+18,1.114953e+18,色玻,2021-12-20T11:42:08.000Z,みんなからの匿名質問を募集中！ こんな質問に答えてるよ ● 消えたいと思った事はありますか… ● 最近買ったものは？… #質問箱 #匿名質問募集中,NaN,NaN,NaN,NaN,NaN,...,NaN,ja,NaN,NaN,0.0,NaN,irh_dso,2021-12-20T11_42_05.000Z_2021-12-21T02_34_19.000Z_9385.csv,1472895087764200000,0.0
8544,1.472895e+18,1.339559e+18,オワリ,2021-12-20T11:42:07.000Z,飛び降りるやつ？ #のびらじ,NaN,NaN,NaN,NaN,NaN,...,NaN,ja,NaN,NaN,0.0,NaN,owowari_44,2021-12-20T11_42_05.000Z_2021-12-21T02_34_19.000Z_9385.csv,1472895083771200000,0.0
8545,1.472895e+18,4.190654e+09,開運絵師（現創展銀賞作家）,2021-12-20T11:42:06.000Z,みんなからの匿名質問を募集中！ こんな質問に答えてるよ ● 死にたいほど辛いとかそういうわ… ● 音ゲーしてる人=オタク これっ… ● 復縁ってありますか？… ● デートするとしたらどこでする？… #質問箱 #匿名質問募集中,NaN,NaN,NaN,NaN,NaN,...,NaN,ja,NaN,NaN,0.0,NaN,atu41oku,2021-12-20T11_42_05.000Z_2021-12-21T02_34_19.000Z_9385.csv,1472895079434300000,0.0


In [80]:
df.query("Correct_label == 1").sample(10).iloc[:, -3:]

,tweetid_short,Label,Correct_label
492,1473105668274300000,1.0,1.0
3460,1472995219260200000,1.0,1.0
2280,1473052746949800000,1.0,1.0
5506,1472940813881800000,1.0,1.0
5643,1472938531526200000,1.0,1.0
2737,1473034437856500000,1.0,1.0
1808,1473067676100900000,1.0,1.0
2864,1473028996678200000,1.0,1.0
873,1473094908055800000,1.0,1.0
6494,1472925349340400000,1.0,1.0


# Check 18W tweets

In [86]:
import pandas as pd

In [117]:
filename = r'T:\Shared drives\T5\Japan_Tweets_2020-01-01_2021-01-01\predicted\positive_all.csv'
# df = pd.read_csv(open(filename, 'rb'), engine='python', encoding='ISO-8859-1', on_bad_lines='skip')
df = pd.read_csv(filename, engine='python', encoding='utf16', on_bad_lines='skip')

# df

In [123]:
for row in df.iloc[34000:34500].sample(60)['cleaned_message']:
    print(row)

私の自傷ダメージ続けても回復しないんすよね。なんでかな。 なんなら会心率も上がってねぇや。
こんなクマに襲われて死にたい 
緩やかに死にたいなーって思う時ない？今私それ
最近死にたい話しかできなくてごめんなさい本当。
価値のある若さと価値の無い若さ ブスだ 死にたいな
死にたい。 
僕ってなんなんだろう。  いなくても全然迷惑にならないだろうな。   そんなんなら早く消えたいよ。
はい死にたい。 この世のすべてがめんどくさい。 死ぬことすらもめんどくさい。 人間為すこと全てかめんどくさい。
もうひとりで家いるのいや死にたい、ひとりすきだけど
昭和5年。親権が父親にしか認められない時代。メンタル壊れ遺書を遺し服毒自殺…。うむぅ…
死にたいっていうか、消えてなくなりたい。 この世に初めからいなかったことにしたい。
自傷行為をしない   #Peing #質問箱 
病んで疲れてくると首吊りたくなるね！
いっそ死にたいなんて思ってるのかな
大事な人を守れなかったから死にたい。
死にたいとか思ってないけどとりあえず人生に疲れた
寝てる間にキス…されなかった…死にたい
イライラし過ぎて禿げて死にそう。 お願いだから禿げないで死にたい
自分を傷つけるな
死にたい僕は今日も息をして 生きたい君は明日を見失って 、、、、みたいな。 
私は消えた方がいい。あと1ヶ月もつか分からない。死にたい。職場で死んだらどうなるかな。
毎日生きるのに疲れたなぁって思ってんのに中々自殺しない自分 中々に頭悪い
最近ストレスがかかるとカジュアルに死にたいって思うようになった
死にたい、虚しい、生きていてもしょうがない
母に過食をバカにされた。殴ってしまった。死にたい。死にたいわ。
早く死にたいよ… 私なんてこの世に居ない方がいい 居たら誰かを不幸にする
死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい
私死にたくはないんだよね、消えたいだけで。
うわあ起きたら怒りが鬱に変わるやつだやばいめっちゃ死にたい😇ｼﾆﾀｲ
親に死にたいって泣きつからたらどうすればいいんや
一思いにセックスして死にたい
涙止まらない死にたいごめんなさい本当に無理です
首を締められたことが何度もあるから首吊りなんてしたくない、死にたくないけど生きていたくない  助

# Tools
## Extract Zhenye's work

In [144]:
import pandas as pd
pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_columns', None)

In [183]:
def fix_data_remove_RT(df):
    df['id'] = df['id'].fillna(0).astype(np.int64).astype(str)
    df['author_id'] = df['author_id'].fillna(0).astype(np.int64).astype(str)
    df['conversation_id'] = df['conversation_id'].fillna(0).astype(np.int64).astype(str)
    df['users_table_id'] = df['users_table_id'].fillna(0).astype(np.int64).astype(str)
    df['users_table_pinned_tweet_id'] = df['users_table_pinned_tweet_id'].fillna(0).astype(np.int64).astype(str)
    df['tweets_table_author_id'] = df['tweets_table_author_id'].fillna(0).astype(np.int64).astype(str)
    df = df[~df['text'].fillna("-").str.startswith("RT").fillna(False)]
    return df

# fix_data(df1)

In [184]:

df1 = pd.read_excel(r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\zhenye_works\ZhenYe_FullRecord.xlsx', sheet_name='_2021-12-10T12_11_05.000Z_9526')
df1 = fix_data_remove_RT(df1)
df2 = pd.read_excel(r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\zhenye_works\ZhenYe_FullRecord.xlsx', sheet_name='_2021-12-12T12_11_05.000Z_9526')
df2 = fix_data_remove_RT(df2)


df3 = pd.read_excel(r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\zhenye_works\ZhenYe_FullRecord.xlsx', sheet_name='2.3W original')
df4 = pd.read_excel(r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\zhenye_works\ZhenYe_FullRecord.xlsx', sheet_name='2.3W postive')

In [185]:
len(df1), len(df2)

(5986, 7819)

In [189]:
df1['INDICATOR'].sum(), df2['INDICATOR'].sum()

(274.0, 778.0)

In [140]:
import numpy as np


,attachments,author_id,conversation_id,created_at,edit_history_tweet_ids,entities,geo,id,in_reply_to_user_id,lang,...,users_table_id,users_table_location,users_table_name,users_table_pinned_tweet_id,users_table_profile_image_url,users_table_protected,users_table_public_metrics,users_table_url,users_table_username,users_table_verified
0,NaN,1.138479e+18,1.469288e+18,2021-12-10T12:49:21.000Z,['1469288125713489920'],NaN,NaN,1469288125713479936,NaN,ja,...,1.138479e+18,ＧＢＮ兵庫,ふしんしゃ〜,1.384809e+18,https://pbs.twimg.com/profile_images/1384811910198030342/_0YGt-ij_normal.jpg,0.0,"{'followers_count': 82, 'following_count': 171, 'tweet_count': 486, 'listed_count': 11}",NaN,husinsya_EXVS,0.0
1,NaN,1.241226e+09,1.469288e+18,2021-12-10T12:49:21.000Z,['1469288125315022850'],NaN,NaN,1469288125315020032,NaN,ja,...,1.241226e+09,NaN,もむお,9.503579e+17,https://pbs.twimg.com/profile_images/657189360182231044/pcJ_qwZu_normal.jpg,0.0,"{'followers_count': 31, 'following_count': 19, 'tweet_count': 17006, 'listed_count': 1}",https://t.co/dKP3IQZhTG,100_100_o,0.0
2,NaN,3.095924e+09,1.469288e+18,2021-12-10T12:49:20.000Z,['1469288121464651782'],"{'hashtags': [{'start': 16, 'end': 23, 'tag': '沈黙の金曜日'}]}",NaN,1469288121464649984,NaN,ja,...,3.095924e+09,NaN,元気,NaN,https://pbs.twimg.com/profile_images/859579944615235584/Nq77v-BF_normal.jpg,0.0,"{'followers_count': 75, 'following_count': 396, 'tweet_count': 12403, 'listed_count': 1}",NaN,Geeeenki1226,0.0
3,NaN,1.193427e+18,1.469288e+18,2021-12-10T12:49:18.000Z,['1469288112094576641'],"{'mentions': [{'start': 3, 'end': 12, 'username': 'hirasawa', 'id': '77907829'}]}",NaN,1469288112094569984,NaN,ja,...,1.193427e+18,きのこの里,たけのこの山ver.U.T.A,NaN,https://pbs.twimg.com/profile_images/1377421300164009984/Sldw9pJ4_normal.jpg,0.0,"{'followers_count': 35, 'following_count': 678, 'tweet_count': 7061, 'listed_count': 0}",NaN,Takenoko_Vlove,0.0
4,NaN,1.164035e+18,1.469202e+18,2021-12-10T12:49:16.000Z,['1469288105190760454'],"{'annotations': [{'start': 47, 'end': 52, 'probability': 0.6566, 'type': 'Person', 'normalized_text': 'もりでるさん'}], 'mentions': [{'start': 0, 'end': 8, 'username': 'riddele', 'id': '187050508'}]}",NaN,1469288105190759936,1.870505e+08,ja,...,1.164035e+18,NaN,日々💎,1.522527e+18,https://pbs.twimg.com/profile_images/1304812506079137793/kltPKn09_normal.png,0.0,"{'followers_count': 668, 'following_count': 135, 'tweet_count': 28936, 'listed_count': 28}",https://t.co/oXiEnukVzw,hi_bibibi,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9487,NaN,3.021205e+09,1.469116e+18,2021-12-10T01:23:46.000Z,['1469115592896815105'],NaN,NaN,1469115592896809984,NaN,ja,...,3.021205e+09,NaN,お茶目な麒麟様bot,NaN,https://pbs.twimg.com/profile_images/578624507843194880/dJoH1pp6_normal.jpeg,0.0,"{'followers_count': 124, 'following_count': 207, 'tweet_count': 51567, 'listed_count': 0}",NaN,bloodygiraffe,0.0
9488,NaN,7.046585e+17,1.468928e+18,2021-12-10T01:23:46.000Z,['1469115589528793089'],"{'mentions': [{'start': 0, 'end': 16, 'username': 'SunshineSmileYB', 'id': '884183488995381248'}]}",NaN,1469115589528790016,8.841835e+17,ja,...,7.046585e+17,日本,Ⓜ️,NaN,https://pbs.twimg.com/profile_images/1370012936102547463/4beDezoj_normal.jpg,0.0,"{'followers_count': 599, 'following_count': 106, 'tweet_count': 49116, 'listed_count': 7}",https://t.co/qZQGkK5OKr,M_emu_emu_,0.0
9489,NaN,1.661866e+08,1.469116e+18,2021-12-10T01:23:45.000Z,['1469115586613772289'],"{'mentions': [{'start': 3, 'end': 13, 'username': 'AyaKadoya', 'id': '1081505660934619136'}]}",NaN,1469115586613769984,NaN,ja,...,1.661866e+08,Tokyo,"HORIO, Naomi 国際フォーカシング研究所認定フォーカシング・プロフェッショナル",8.161181e+17,https://pbs.twimg.com/profile_images/1456084004835565570/aSSRdj4v_normal.jpg,0.0,"{'followers_count': 983, 'following_count': 719, 'tweet_count': 9874, 'listed_count': 23}",https://t.co/6weKqL8VJo,hn_Focusing,0.0
9490,NaN,1.579172e+08,1.469115e+18,2021-12-10T01:23:04.000Z,['1469115415943315463'],"{'mentions': [{'start': 3, 'end': 16, 'username': 'babienobitch'

In [150]:
df1.columns

Index(['attachments', 'author_id', 'conversation_id', 'created_at',
       'edit_history_tweet_ids', 'entities', 'geo', 'id',
       'in_reply_to_user_id', 'lang', 'lat', 'lon', 'media_table_height',
       'media_table_media_key', 'media_table_rows', 'media_table_type',
       'media_table_url', 'media_table_width', 'place_id',
       'places_table_country', 'places_table_country_code',
       'places_table_full_name', 'places_table_geo', 'places_table_id',
       'places_table_name', 'places_table_place_type',
       'polls_table_duration_minutes', 'polls_table_end_datetime',
       'polls_table_id', 'polls_table_options', 'polls_table_rows',
       'possibly_sensitive', 'public_metrics', 'referenced_tweets',
       'reply_settings', 'INDICATOR', 'source', 'text',
       'tweets_table_attachments', 'tweets_table_author_id',
       'tweets_table_conversation_id', 'tweets_table_created_at',
       'tweets_table_edit_history_tweet_ids', 'tweets_table_entities',
       'tweets_table_geo'

In [354]:
keywords = ['首吊り', '首を吊る', '首つり', '死ぬ気', '自分を傷つける', 'この世を去る', '死ぬに値する', '自分の人生を終わらせたいという願望', '死にたい', '自傷', '私の命を奪う', '死にたい', '死にたいです', '私の遺書', '私の人生を終わらせる', '決して起きない', '生きる価値がない', '飛び降りる', '永遠に眠る', '電車に飛び込む', '私がいないほうがいい', '生きるのに疲れた', '一人で死ぬ', '永遠に眠る', '私の悲しい人生', 'ストレスを感じる', 'ストレスで参っている', '感情の起伏が激しい', '私自身が嫌い', '精神的に弱い', '練炭', '焼身', '服毒', 'もう死にたい', '自殺サイト楽に死ねる方法', '生きることがつらい', '死にたい 助けて', '安楽死方法', '一番楽に死ねる方法', '簡単に死ねる方法', '消えたい', '確実に死ねる方法', '生きる意味が分からない', 'うつ 死にたい']
pattern = '|'.join(keywords)  #"自殺"  "自殺", 

df = pd.DataFrame([{'message':"自殺 keywords  "}, {'message':"keywords"}, {'message':"自殺"}, {'message':"自分を傷つける"}, {'message':"自殺死ぬに値する"}])
df['Label'] = 1
keywords_idx = df['message'].str.contains(pattern)
sucide_idx = df['message'].str.contains('自殺')
mask_idx = (~keywords_idx) & sucide_idx
mask_idx
df.loc[mask_idx, "Label"] = 0
df

,message,Label
0,自殺 keywords,0
1,keywords,1
2,自殺,0
3,自分を傷つける,1
4,自殺死ぬに値する,1


## Correct message

In [335]:
def _correct_label(df, correct_df, text_col="message", label_col="Correct_label"):    
    merged_df = df.merge(correct_df, left_on='message', right_on='message', how='left')
    # print(df)
    mask_idx = merged_df['Correct_label'].isna()
    print("Label not nan count:", (~mask_idx).sum(), len(merged_df))
    merged_df.loc[~mask_idx, 'Label'] = merged_df[~mask_idx]['Correct_label'].to_list()

    return merged_df

def correct_label(df):
    df1 = pd.read_csv(r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\bi_label_in_test_correct_labels.csv')
    df2 = pd.read_csv(r'T:\Shared drives\T5\Japan_Tweets_2021_01_01_2022_01_01\bi_label_in_train_correct_labels.csv')
    correct_df = pd.concat([df1, df2]).drop_duplicates()
    correct_df['message'] = correct_df.apply(clean_twt_row, axis=1)
    correct_df = correct_df.drop_duplicates()
    # correct_df = correct_df.rename(columns={'Label': "Correct_label"})
    
    return correct_df
    
    df = _correct_label(df, correct_df=correct_df, text_col="message", label_col="Correct_label")
    df = df.drop(columns=['Correct_label'])
    
    return df

positive_idx = samples_df2['RISK(YES:1,NO:0)'] == 1
samples_df2_positive = samples_df2[positive_idx][['id', 'text']]
samples_df2_positive['message'] = samples_df2_positive['text']
samples_df2_positive = samples_df2_positive.drop(columns=['text', 'id'])

# negative in samples_df2
samples_df2_negative = samples_df2[~positive_idx][['id', 'text']]
samples_df2_negative['message'] = samples_df2_negative['text']
samples_df2_negative = samples_df2_negative.drop(columns=['text', 'id']) 
    
samples_df2_positive['message'] = samples_df2_positive.apply(clean_twt_row, axis=1)    
   
samples_df2_positive['Label'] = 1
samples_df2_negative['Label'] = 0   

df = correct_label(df=samples_df2_positive)
df

,Correct_label,message
0,1,#嫌な上司あるある 急な出勤、急な休日。 「何であなたに合わせなくてはいけな #上司 嫌い 社会人 給与 学生 死にたい 上手くやっていけるのか 憂鬱 社畜
1,0,"1度に30錠以上飲んだり自傷も沢山した 今思い返すとほんとバカだったなぁと思う(笑) 薬を雑に扱ってきた分 これからは、""薬""を正しく使い また提供する人間として、一人でも多くの人の健康・生活を支えられるよう頑張ろうと思います:relieved_face:"
2,1,【また学校休んだ。罪悪感しかない。行ったら苦しいし辛い。はやく死にたい】 ただ怖い。今は全部が怖い。 部屋から出るのが怖い。 はやく死にたいな。 学校なんてなければいいのにな。 (続きを読む⇒)
3,0,【定期ポスト】不適切な表現をすると、私は、頭が残念な人のレベルまでしか、英語、中国語、スペイン語を話せないんだろうな。なんて低脳だ・・・馬鹿にしては頭がマシなレベルを維持できるようがんばるけど、死にたいわ。
4,1,あ、また「死にたい」って言っちゃった 死にたい.........
...,...,...
1079,0,電車の中で淫夢CD聞いてて、目的駅について立ちあがった瞬間イヤホンはずれて、野獣先輩が大衆の面前で「イキすぎィ！イクゥ！イクイクイク…ンアーッ！」って言い出した時。でももうこれは「死にてえｗｗｗｗ」とかじゃない「死にたい」
1081,0,あぁ…もう、バタフライスツールにはさまれて死にたい
1082,0,生き物ってほんとに切ない。 なんでいつかは死ぬんだろうね なんで老いていくんだろうね。 死にたくないし老いたくない。 ふと死んだ時のこととか考えて 悲しくなる。病んだ時は死にたいって 思うのに通常時ふと死ぬ時のこと 考えると死にたくないって涙出る 情緒不安定すぎないか
1083,1,突然涙出てきた無理死にたい苦しい生きているのが辛い


In [333]:
df['Correct_label'].value_counts()

1    671
0    395
Name: Correct_label, dtype: int64

In [334]:

df.groupby(['message']).agg(positive_sum=('Correct_label', sum),
                            tweet_cnt=('message', 'count'),
                           ).sort_values('tweet_cnt')[-3:] 
# df.dtypes

,positive_sum,tweet_cnt
message,,
何か嫌なことがあったとき、 もう全部やだ。自分が死にたい。 って思う人と もう全部やだ。全員ぶっ殺す。 では世界線が違うし、 私は前者だったけどここ数年色々ありすぎて後者になりました。 まる。,1,1
今日死ぬ気がします #BadBuddySeriesEP7,1,1
･･･死にたい,1,1


## Estimate positive rate

In [346]:
import glob
csv_dir = r'T:\Shared drives\T5\Japan_Tweets_2020-01-01_2021-01-01\predicted\*.csv'
csv_files = glob.glob(csv_dir)[:-1]
print("Found files:", len(csv_files))
csv_files[-1]

Found files: 159


'T:\\Shared drives\\T5\\Japan_Tweets_2020-01-01_2021-01-01\\predicted\\_2020-12-31T09_50_44.000Z_9649.csv'

In [343]:
df = pd.concat([pd.read_csv(f) for f in csv_files])
df

,tweetid,userid,username,postdate,message,geoType,longitude,latitude,place,placeBboxwest,placeBboxeast,placeBboxsouth,placeBboxnorth,source,usermentions,urls,hashtags,retweetCount,userlocation,followerscount,friendscount,joinday,favouritesCount,language,statusescount,replytostatusid,replytoUserid,userverified,userdescription,userurl,favoritecount,listedcount,placetype,bboxtype,placeid,year,month,geo,country_code,country,tweet_lang,message_en,message_cn,sentiment,topic,username_account,cleaned_message,len,trained_model_all,final_label
0,1.308966e+18,7.226274e+17,ょ,2020-09-24T03:06:43.000Z,消えたい,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Twitter for iPhone,NaN,NaN,NaN,0.0,お部屋,22.0,NaN,2016-04-20T03:26:05.000Z,NaN,NaN,NaN,NaN,NaN,False,天魔@___Noo00N,NaN,NaN,1.0,NaN,NaN,NaN,2020.0,9.0,NaN,NaN,NaN,ja,NaN,NaN,0,NaN,kxeexdx_0lIl,消えたい,4,1,1
1,1.308966e+18,1.718268e+09,死にたいbot,2020-09-24T03:04:55.000Z,死にたい,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,twittbot.net,NaN,NaN,NaN,0.0,NaN,231.0,NaN,2013-09-01T08:03:20.000Z,NaN,NaN,NaN,NaN,NaN,False,ひたすら死にたいと呟くbot 死にたくなったらRTしてみてはいかがでしょうか,NaN,NaN,0.0,NaN,NaN,NaN,2020.0,9.0,NaN,NaN,NaN,ja,NaN,NaN,0,NaN,shinitainodesu,死にたい,4,1,1
2,1.308963e+18,1.020290e+18,隕石K,2020-09-24T02:56:12.000Z,死にたい,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Twitter for iPhone,NaN,NaN,NaN,1.0,お絵描きアカウントはリンク先,646.0,NaN,2018-07-20T12:51:39.000Z,NaN,NaN,NaN,NaN,NaN,False,ナマコブシ リンカ ぐんまちゃん SIHanatsuka,https://t.co/DGPC7zsGsy,NaN,24.0,NaN,NaN,NaN,2020.0,9.0,NaN,NaN,NaN,ja,NaN,NaN,0,NaN,K46953424,死にたい,4,1,1
3,1.308962e+18,3.119633e+08,さ,2020-09-24T02:51:26.000Z,死にたい,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Twitter for iPhone,NaN,NaN,NaN,0.0,NaN,1205.0,NaN,2011-06-06T11:00:53.000Z,NaN,NaN,NaN,NaN,NaN,False,アラサーが選ぶ「本当に買ってよかったコスメ」10選,NaN,NaN,43.0,NaN,NaN,NaN,2020.0,9.0,NaN,NaN,NaN,ja,NaN,NaN,0,NaN,salamander_46,死にたい,4,1,1
4,1.308962e+18,1.305451e+18,こ み💜ハヌルと一緒,2020-09-24T02:50:46.000Z,死にたい,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Twitter for iPhone,NaN,NaN,NaN,0.0,NaN,703.0,NaN,2020-09-14T10:20:14.000Z,NaN,NaN,NaN,NaN,NaN,False,ゲームとコスメとBTSが好き1/15からハヌル（ペキニーズ♂）と二人暮らし。育犬アカウントと化しつつある。ゲーム→原神崩壊3rdメイン。FGOとウマ娘まったり復帰しました コスメ→エレガンス、ルナソル ディオール多め BTSのテテに夢中💜共通の趣味の方仲良くしてください😊,NaN,NaN,5.0,NaN,NaN,NaN,2020.0,9.0,NaN,NaN,NaN,ja,NaN,NaN,0,NaN,L8EA6uDy7FPrTmH,死にたい,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9644,1.344551e+18,5.509766e+08,三宅香帆💐『妄想古文』10/27発売！,2020-12-31T07:47:40.000Z,#2020年に読んだ本ベスト10冊 7. 『大奥』(1-18巻、よしながふみ) 政治と疫病、今と重なりすぎる。 6. 『鬱屈精神科医、占いにすがる』(春日武彦) 鬱や自傷を考えるきっかけに。 5. 『A子さんの恋人』(7巻、近藤聡乃) 圧巻の最終巻！ 4. 『海街diary』(1-9巻、吉田秋生) 日本文学の後継を見た感。 https://t.co/G1kBea6Pz5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Twitter for iPhone,NaN,https://twitter.com/m3_myk/status/1344550777440796674/photo/1;https://twitter.com/m3_myk/status/1344550777440796674/photo/1;https://twitter.com/m3_myk/status/1344550777440796674/photo/1;https://twitter.com/m3_myk/status/1344550777440796674/photo/1;https://pbs.twimg.com/media/EqjNzYhUYAAFcna.jpg;https://pbs.twimg.com/media/EqjNzYfVgAAtByO.jpg;https://pbs.twimg.com/media/EqjNzbsVoAA7IjO.jpg;https://pbs.twimg.com/media/EqjNzbsVgAIrSIY.jpg,2020年に読んだ本ベスト10冊,3.0,連載一覧、ブログURL、過去実績はこちら▽,12941.0,NaN,2012-04-11T13:07:35.000Z,NaN,NaN,NaN,NaN,NaN,False,書評家、作家。■『(萌えすぎて)絶対忘れない！妄想古文』10/27発売🌸 ■本について喋るyoutube https://t.co/rhWlKWbzJF ■著書『文芸オタクの私が教える バズる文章教室』等7冊。■本と漫画と宝塚と女子アイドルのおたく。kaho.haru2012@gmail.com,https://t.co/02Kzrot1pT,NaN,150.0,NaN,NaN,NaN,2020.0,12.0,NaN,NaN,NaN,ja,NaN,NaN,0,NaN,m3_myk,#2020年に読んだ本ベスト10冊 7. 『大奥』(1-18巻、よしながふみ) 政治と疫病、今と重なりすぎる。 6. 『鬱屈精神科医、占いにすがる』(春日武彦) 鬱や自傷を考えるきっかけに。 5. 『A子さんの恋人』(7巻、近藤聡乃) 圧巻の最終巻！ 4. 『海街diary』(1-9巻、吉田秋生) 日本文学の後継を見た感。,191,0,0
9645,1.344466e+18,9.601237e+17,フーマリ,2020-12-31T02:12:34.000Z,@mori_marino 『アマネギムナジウム』（7巻で完結）https://t.co/l2sBHaMrQh 自分で作ったお人形が動いた話 『origin』（10巻で完結）https://t.co/z6pG7BOTGh ロボットの話。出てくる天才の女の子が可愛い 『青野くんに触りたいから死にたい』（未完結、今7巻）https://t.co/mdJr8YcHXK 彼氏が幽霊になる話。,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T

In [344]:
positive_df = df.query("final_label == 1")
positive_df

,tweetid,userid,username,postdate,message,geoType,longitude,latitude,place,placeBboxwest,placeBboxeast,placeBboxsouth,placeBboxnorth,source,usermentions,urls,hashtags,retweetCount,userlocation,followerscount,friendscount,joinday,favouritesCount,language,statusescount,replytostatusid,replytoUserid,userverified,userdescription,userurl,favoritecount,listedcount,placetype,bboxtype,placeid,year,month,geo,country_code,country,tweet_lang,message_en,message_cn,sentiment,topic,username_account,cleaned_message,len,trained_model_all,final_label
0,1.308966e+18,7.226274e+17,ょ,2020-09-24T03:06:43.000Z,消えたい,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Twitter for iPhone,NaN,NaN,NaN,0.0,お部屋,22.0,NaN,2016-04-20T03:26:05.000Z,NaN,NaN,NaN,NaN,NaN,False,天魔@___Noo00N,NaN,NaN,1.0,NaN,NaN,NaN,2020.0,9.0,NaN,NaN,NaN,ja,NaN,NaN,0,NaN,kxeexdx_0lIl,消えたい,4,1,1
1,1.308966e+18,1.718268e+09,死にたいbot,2020-09-24T03:04:55.000Z,死にたい,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,twittbot.net,NaN,NaN,NaN,0.0,NaN,231.0,NaN,2013-09-01T08:03:20.000Z,NaN,NaN,NaN,NaN,NaN,False,ひたすら死にたいと呟くbot 死にたくなったらRTしてみてはいかがでしょうか,NaN,NaN,0.0,NaN,NaN,NaN,2020.0,9.0,NaN,NaN,NaN,ja,NaN,NaN,0,NaN,shinitainodesu,死にたい,4,1,1
2,1.308963e+18,1.020290e+18,隕石K,2020-09-24T02:56:12.000Z,死にたい,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Twitter for iPhone,NaN,NaN,NaN,1.0,お絵描きアカウントはリンク先,646.0,NaN,2018-07-20T12:51:39.000Z,NaN,NaN,NaN,NaN,NaN,False,ナマコブシ リンカ ぐんまちゃん SIHanatsuka,https://t.co/DGPC7zsGsy,NaN,24.0,NaN,NaN,NaN,2020.0,9.0,NaN,NaN,NaN,ja,NaN,NaN,0,NaN,K46953424,死にたい,4,1,1
3,1.308962e+18,3.119633e+08,さ,2020-09-24T02:51:26.000Z,死にたい,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Twitter for iPhone,NaN,NaN,NaN,0.0,NaN,1205.0,NaN,2011-06-06T11:00:53.000Z,NaN,NaN,NaN,NaN,NaN,False,アラサーが選ぶ「本当に買ってよかったコスメ」10選,NaN,NaN,43.0,NaN,NaN,NaN,2020.0,9.0,NaN,NaN,NaN,ja,NaN,NaN,0,NaN,salamander_46,死にたい,4,1,1
4,1.308962e+18,1.305451e+18,こ み💜ハヌルと一緒,2020-09-24T02:50:46.000Z,死にたい,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Twitter for iPhone,NaN,NaN,NaN,0.0,NaN,703.0,NaN,2020-09-14T10:20:14.000Z,NaN,NaN,NaN,NaN,NaN,False,ゲームとコスメとBTSが好き1/15からハヌル（ペキニーズ♂）と二人暮らし。育犬アカウントと化しつつある。ゲーム→原神崩壊3rdメイン。FGOとウマ娘まったり復帰しました コスメ→エレガンス、ルナソル ディオール多め BTSのテテに夢中💜共通の趣味の方仲良くしてください😊,NaN,NaN,5.0,NaN,NaN,NaN,2020.0,9.0,NaN,NaN,NaN,ja,NaN,NaN,0,NaN,L8EA6uDy7FPrTmH,死にたい,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
867,1.344341e+18,1.337457e+18,🖕,2020-12-30T17:54:20.000Z,今出れば死ぬほど稼げるんだろうなと思うと一分一秒死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Twitter for iPhone,NaN,NaN,NaN,0.0,話しかけてこないでください😅💦💦,3.0,NaN,2020-12-11T17:59:00.000Z,NaN,NaN,NaN,NaN,NaN,False,｢一部の｣人間ほんと気持ち悪い,NaN,NaN,0.0,NaN,NaN,NaN,2020.0,12.0,NaN,NaN,NaN,ja,NaN,NaN,0,NaN,pompompig_,今出れば死ぬほど稼げるんだろうなと思うと一分一秒死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい死にたい,140,1,1
868,1.344492e+18,2.323352e+09,レナ•デブ•ジ•エンド🐹,2020-12-31T03:53:33.000Z,末期すぎる、、、 上司のせいでほんとに 今まで我慢我慢してたのに人間まで否定されててここ数日外にも出れません食欲もありませんひたすらベッドの中で消えたいって、、、安定剤と睡眠剤に頼ってばかり。人にも会いたくないし人の幸せ見ると死にたくなるのでSNS全て切りました。私の人生どうなるの。,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Twitter for iPhone,NaN,NaN,NaN,0.0,next→,168.0,NaN,2014-02-02T06:55:21.000Z,NaN,NaN,NaN,NaN,NaN,False,また会えた時に軌跡の続きをしようよ,https://t.co/XIPnwZ94bp,NaN,0.0,NaN,NaN,NaN,2020.0,12.0,NaN,NaN,NaN,ja,NaN,NaN,0,NaN,ds1102s2r,末期すぎる、、、 上司のせいでほんとに 今まで我慢我慢してたのに人間まで否定されててここ数日外にも出れません食欲もありませんひたすらベッドの中で消えたいって、、、安定剤と睡眠剤に頼ってばかり。人にも会いたくないし人の幸せ見ると死にたくなるのでSNS全て切りました。私の人生どうなるの。,142,1,1
869,1.344481e+18,1.340892e+18,、,2020-12-31T03:11:35.000Z,でもこの垢始めるまではまじで鬱酷すぎて、 部屋でずっと横たわって時間が過ぎるのを待ってて、 ずっと頭に嫌な考えがよぎるからそれを止めるように自傷行為して…みたいなのをずっと繰り返してたんよ。 だから昨日何やった？一昨日は？ってマジで記憶がなくて、 自分って生きてんのかなって思ってた。,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Twitter for iPad,NaN,NaN,NaN,0.0,NaN,1.0,NaN,2020-12-21T05:30:36.000Z,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2020.0,12.0,NaN,Na

In [349]:
raw_retweet_cnt = df['message'].str.startswith("RT").sum()
print("positive_retweet_cnt count:", raw_retweet_cnt)


positive_retweet_cnt count: 332337


In [350]:
positive_retweet_cnt = positive_df['message'].str.startswith("RT").sum()
print("positive_retweet_cnt count:", positive_retweet_cnt)


positive_retweet_cnt count: 1821


In [353]:
positive_cnt = len(positive_df) - positive_retweet_cnt
raw_cnt_no_RT = len(df) - raw_retweet_cnt
positive_rate = positive_cnt*0.88 / raw_cnt_no_RT
print("Positive rate:", positive_rate)


Positive rate: 0.1310506175327088
